In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse

#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1612188940752_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:


#Query some reviews
query_1 = "select distinct r.ugc_id,  mg.name, r.review_headline, r.review_comments, \
case when (contains(r.observations, 'CR') )  then 1 else 0 end as CR \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where r.locale in ('en_US','en_GB','en_CA') and contains(r.observations, 'PM') and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and r.created_date > '2020-01-01' \
and mg.usage_type != 'Test' \
and ugc_id in (277427048, 277425575) \
"

#and contains(r.observations, 'CR') \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   UGC_ID           2 non-null      int64 
 1   NAME             2 non-null      object
 2   REVIEW_HEADLINE  2 non-null      object
 3   REVIEW_COMMENTS  2 non-null      object
 4   CR               2 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 208.0+ bytes

In [16]:
reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID  ... CR
0  277425575  ...  0
1  277427048  ...  0

[2 rows x 5 columns]

In [17]:
reviews['ALL_TEXT'] =  reviews['REVIEW_HEADLINE'] + ' '+reviews['REVIEW_COMMENTS']

reviews['NAME_1ST_PART'] = reviews['NAME'].apply(lambda x: x.split()[0])
reviews['NAME_1ST_PART_LOWER'] = reviews['NAME_1ST_PART'].str.lower()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#dfTest['bar'] = dfTest.apply(lambda row : row['foo'].replace(str(row['date']), ''), axis=1)
reviews['ALL_TEXT_2'] = reviews.apply(lambda x: x['ALL_TEXT'].replace(str(x['NAME_1ST_PART']),''), axis = 1)
reviews['ALL_TEXT_3'] = reviews.apply(lambda x: x['ALL_TEXT_2'].replace(str(x['NAME_1ST_PART_LOWER']),''), axis = 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID                      NAME  \
0  277425575  Wickes Building Supplies   
1  277427048  Wickes Building Supplies   

                          REVIEW_HEADLINE  \
0  Last time I order anything from Wickes   
1  Decking screws from Wickes               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      REVIEW_COMMENTS  \
0  Having waited weeks for this delivery, I received a text message that it had been delivered and signed for by me at 18.51. This is a lie. I have not had a delivery and have not signed anything. God

In [7]:
 
#import pandas as pd 
#data =[['Went to Amazon.','Amazon'],['Love to shop at Amazon, Target, Walmart.', 'Amazon']]
#df = pd.DataFrame(data, columns = ['ALL_TEXT', 'NAME']) 
#df 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#df['AMZN_TGT'] = df['ALL_TEXT'].str.extractall(r'\b(Amazon|Target|Walmart)\b').unstack().apply(lambda x:','.join(x.dropna()), axis=1)
#df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#df['not_client']=df['AMZN_TGT'].str.lower().ne('amazon')
#df['NAME']=df['NAME'].str.lower()
#df['not_client']=df['AMZN_TGT'].str.lower().ne(df['NAME'])
#df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:

#Waitrose is owned by John Lewis - so shouldn't be marked for John Lewis - add to regex list or no?
#Remove john lewis, big lots
#Remove Goodwill - doesn't count as retailer
#If on or on- before target then ignore, lots of references to products being on target for size or whatever and not refering to retailer Target
#(?:(?<!ON[ -]))TARGET
reviews['regex_retailer_insens_case'] = reviews['ALL_TEXT'].str.extractall(r'\b(AMAZON(?! (?:PAY|ALEXA|FIRE STICK))|E[ -]?BAY|OVERSTOCK|WAL[ -]?MART|LOWE\'?S|HOME DEPOT|MENARD\'?S|B ?& ?Q|WICKES|CABELA\'?S|FLEET FARM|DOLLAR TREE|DOLLAR GENERAL|FAMILY DOLLAR|SAM\'?S|COSTCO|MACY\'?S|NORDSTROM|JC[ ]?PENNE?Y|TJ[ -]?MAXX?|MARSHALL\'?S|(?:(?<!BOB ))ROSS|KOHL\'?S|J[ ]?CREW|CVS|WALGREEN\'?S|RITE[ -]?AID|QVC|HSN|KROGER|WHOLE FOODS|JEWEL OSCO|TJ\'?S|TRADER JOE\'?S?|WEGMAN\'?S|ALBERTSON|ALDI\'?S?|TESCO(?:(?! (?:CLUB[ ]?CARD|POINTS|VOUCHERS)))|CARREFOUR|SEPHORA|POTTERY BARN|IKEA|BED BATH & BEYOND|BED BATH AND BEYOND|WAYFAIR|ZAPPOS|FOOT LOCKER|DSW|AUTO[ ]?ZONE|ETSY|HOBBY LOBBY|MICHAEL\'?S|BLICK|DICK\'?S|REI|AETNA|STATE FARM|IPSY)\b', flags=re.I).unstack().apply(lambda x:','.join(x.dropna()), axis=1)
#.str.extract(r'([^@A-Z][-A-Z0-9:%_\+~#=]+\.(?:CO|COM|NET|ORG|GOV|EDU|APP|IN|BE|GL|NL|LY|FR)\b)', flags=re.I)

#Require case sensitivity
#Ace, Target, 
#Otherwise - ace the word, target as in target practice, on target, etc, heb is a dutch word
reviews['regex_retailer_sens_case'] = reviews['ALL_TEXT'].str.extractall(r'\b(Ace|Sears|Target|HEB)\b').unstack().apply(lambda x:','.join(x.dropna()), axis=1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Merge case insensitive and sensitive columns

#df['c'] = np.where(df["a"].isnull(), df["b"], df["a"] )
reviews['regex_retailer'] = np.where(reviews['regex_retailer_insens_case'].isnull(),reviews['regex_retailer_sens_case'], reviews['regex_retailer_insens_case'] )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#regex_retailer returns nan from above if no match
#Need to fill in with random string in order to check for name match



reviews['regex_retailer'] = reviews['regex_retailer'].fillna(reviews['NAME_1ST_PART'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
reviews['NAME_1ST_PART'] = reviews['NAME_1ST_PART'].str.lower()
reviews['regex_retailer'] = reviews['regex_retailer'].str.lower()

#Returns True if there is a retailer mention that is not the client name
#False if no retailer mention (i.e. zzzzzzzzzz)
#False if retailer mention = client name
#reviews['retailer_not_client'] = reviews.apply(lambda x: x.NAME_1ST_PART not in x.regex_retailer if x.regex_retailer!='zzzzzzzzzz' else False, axis=1)



#df['not_client']=df['AMZN_TGT'].str.lower().ne(df['NAME'])
reviews['retailer_not_client']=reviews['regex_retailer'].ne(reviews['NAME_1ST_PART'])
#reviews['retailer_not_client'] = reviews.apply(lambda x: x.regex_retailer.ne(x.NAME_1ST_PART) if x.regex_retailer!='zzzzzzzzzz' else False, axis=1)


#Trying to look through list like "Amazon, Target" and check if anything besides client name is there
#So if client is Amazon, returns True for "Amazon, Target" but false for just "Amazon"
#if (reviews['regex_retailer'] != 'zzzzzzzzzz') :
#    reviews['retailer_not_client'] = reviews['regex_retailer'].ne(reviews['NAME_1ST_PART'])
#else:
#    reviews['retailer_not_client'] = False


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
reviews.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      UGC_ID           NAME                         REVIEW_HEADLINE  \
0  274567494  Plow & Hearth  good product                             
1  274756585  Plow & Hearth  Yes                                      
2  274577907  Plow & Hearth  Beautiful!                               
3  274587948  Plow & Hearth  Would not buy again                      
4  274588107  Plow & Hearth  Would not buy again.                     
5  274611148  Plow & Hearth  Perfect Trio                             
6  274443961  Plow & Hearth  yes                                      
7  274478330  Plow & Hearth  I would recommend this                   
8  274584011  Plow & Hearth  I buy this product when needed           
9  274612207  Plow & Hearth  Measure your bricks for a perfect fit!   

                                                                                                                                                                                             REVIEW_COMMENTS  \
0  Product

In [18]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#test = reviews[reviews['regex_retailer']==False]
#test.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<string>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.

In [19]:
reviews['model_CR'] = 0

reviews.loc[(reviews['retailer_not_client']==True ), 'model_CR'] = 1

#Ignore where midwayusa client name bc target references don't refer to Target the retailer but target as in target practice
#Can't look for case sensitivity bc people still start a sentence with target and therefore it's capitalized
#Midsouth Shooters as well, Cheaper Than Dirt as well
#3Rivers Archery, JoeBobOutfitters - Terminated, Leupold - TERMINATED 
reviews.loc[((reviews['NAME']=='MidwayUSA') |(reviews['NAME']=='Midsouth Shooters') | (reviews['NAME']=='Cheaper Than Dirt')| (reviews['NAME']=='3Rivers Archery')| (reviews['NAME']=='JoeBobOutfitters - Terminated')| (reviews['NAME']=='Leupold - TERMINATED'))&(reviews['regex_retailer']=='target'), 'model_CR'] = 0
#Ignore where Citi card holders mention Costco bc it's a Costco related credit card
reviews.loc[(reviews['NAME']=='Citigroup Inc.') &(reviews['regex_retailer']=='costco'), 'model_CR'] = 0
#Ignore where sears in mentioned in BBQ Guys reviews, almost always refering to searing of meat
reviews.loc[(reviews['NAME']=='BBQ Guys') &(reviews['regex_retailer']=='sears'), 'model_CR'] = 0
#Marshalls is a brand sold at Wickes, also at Guitar Center, and at Musician's Friend, Sam Ash
#Books-A-Million has series with Marshalls in the name
reviews.loc[((reviews['NAME']=='Wickes Building Supplies')|(reviews['NAME']=='Guitar Center')|(reviews['NAME']=="Musician's Friend")|(reviews['NAME']=='Sam Ash')|(reviews['NAME']=='Books-A-Million')) &((reviews['regex_retailer']=='marshalls')|(reviews['regex_retailer']=="marshall's")), 'model_CR'] = 0
#Dick Blick
reviews.loc[(reviews['NAME']=='Dick Blick') &(reviews['regex_retailer']=='blick'), 'model_CR'] = 0
#Consumer Reports allows retailer mentions so no CR label
reviews.loc[(reviews['NAME']=='Consumer Reports'), 'model_CR'] = 0


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
FP = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 1)]
FN = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 0)]
TP = reviews[(reviews['CR'] == 1) & (reviews['model_CR'] == 1)]
TN = reviews[(reviews['CR'] == 0) & (reviews['model_CR'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10792

In [22]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12561

In [23]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

28702

In [24]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7899622

In [25]:
#FN.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
FN[['NAME','UGC_ID']].groupby(['NAME']).count().reset_index().sort_values(['UGC_ID'], ascending=False).head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         NAME  UGC_ID
719  Ulta                      669   
323  John Lewis & Partners     386   
660  The Hartford - P&C        341   
35   Art.com - TERMINATED      274   
13   Advance Auto Parts        255   
754  Wickes Building Supplies  251   
124  Cheaper Than Dirt         243   
128  Citigroup Inc.            238   
173  Discount Tire             230   
643  TSC                       174

In [27]:
check=FP[FP['NAME']=='Wickes Building Supplies']
check.head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID                      NAME  \
6176   298784319  Wickes Building Supplies   
6180   298769417  Wickes Building Supplies   
10275  268936291  Wickes Building Supplies   
13188  276869568  Wickes Building Supplies   
37761  317659440  Wickes Building Supplies   
37765  317500678  Wickes Building Supplies   
44190  277427048  Wickes Building Supplies   
44351  277396076  Wickes Building Supplies   
44381  277425575  Wickes Building Supplies   
44472  277414019  Wickes Building Supplies   

                                                    REVIEW_HEADLINE  \
6176   OK Castors                                                     
6180   Great product great price and no delay for delivery can coll   
10275  Wickes need to learn the correct way to store wood             
13188  Incomplete delivery and no follow-up from Wickes               
37761  Well done Wickes own brand                                     
37765  Paint                                                       

In [41]:
FP.groupby(['regex_retailer']).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                                UGC_ID  NAME  \
regex_retailer                                                                 
ace                                                             222     222    
ace,ace                                                         35      35     
ace,ace,ace                                                     2       2      
ace,ace,ace,ace                                                 1       1      
aetna                                                           16      16     
aetna,aetna                                                     1       1      
aetna,aetna,aetna                                               3       3      
aetna,aetna,aetna,aetna                                         1       1      
albertson                                                       10      10     
aldi                                                            67      67     
aldi's                                  

In [23]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("regex_retailer_insens_case", StringType(), True)\
,StructField("regex_retailer_sens_case", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FN = spark.createDataFrame(FN, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
FN.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FN_test3").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…